---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [35]:
import re
import numpy as np

#date_norm = df.to_frame(name="Original text")


def date_sorter():
    
    # Your code here
    date_norm = df.to_frame(name="Original text")
    date_norm["Parsed date"] = None
    
    # Drom specific to less specific, do not consider already matched
    #date_norm["Parsed date"] = date_norm["Original text"].apply(lambda x: re.findall(r"\d{4}", x))
    date_norm["Parsed date"] = date_norm["Original text"].apply(lambda x: re.findall(r"\d{1,2}[/-]\d{1,2}[/-]\d{2,4}", x))
    
    #date_norm["Parsed date"] = date_norm.loc[pd.isnull(date_norm["Parsed date"]), "Original text"].apply(lambda x: str(re.findall(r"\d{4}", x)))
    #date_norm["Parsed date"] = date_norm[["Original text", "Parsed date"]].apply(
    #                                lambda row: re.findall(r"\d{4}", row["Original text"]) if pd.isnull(row["Parsed date"]).any() else row["Parsed date"], axis=1)
    
    # Not yet working
    #date_norm["Parsed date"] = date_norm[["Original text", "Parsed date"]].apply(
                                    #lambda row: re.findall(r"(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}, )\?\d{4}", row["Original text"]) 
     #                               if row["Parsed date"] == [] else row["Parsed date"], axis=1)
    date_norm["Parsed date"] = date_norm[["Original text", "Parsed date"]].apply(
                                    lambda row: re.findall(r"(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2})?,?\W?\d{4}", 
                                                           row["Original text"]) 
                                                            if row["Parsed date"] == [] else row["Parsed date"], axis=1)
    
    
    date_norm["Parsed date"] = date_norm[["Original text", "Parsed date"]].apply(
                                    lambda row: re.findall(r"(\d{1,2}[/-]\d{4})[^\d]", row["Original text"]) if row["Parsed date"] == [] else row["Parsed date"], axis=1)
   
    
    
    date_norm["Parsed date"] = date_norm[["Original text", "Parsed date"]].apply(
                                    lambda row: re.findall(r"[^\d](\d{4})[^\d]", row["Original text"]) if row["Parsed date"] == [] else row["Parsed date"], axis=1)
    
    # Year e.g. at the beg of line
    date_norm["Parsed date"] = date_norm[["Original text", "Parsed date"]].apply(
                                    lambda row: re.findall(r"(\d{4})", row["Original text"]) if row["Parsed date"] == [] else row["Parsed date"], axis=1)
    
    
    
    # Put to datetime, to sort
    date_norm["Parsed date 2"] = date_norm["Parsed date"].apply("".join)
    
    # Typo fixing
    date_norm["Parsed date 2"] = date_norm["Parsed date 2"].str.replace("Janaury", "January")
    date_norm["Parsed date 2"] = date_norm["Parsed date 2"].str.replace("Decemeber", "December")
    
    date_norm.iloc[72]= "07-11-1977"
    
    date_norm["Parsed date 2"] = pd.to_datetime(date_norm["Parsed date 2"], errors="coerce", dayfirst=False)
    
    
    
    return date_norm.sort_values(by="Parsed date 2", ascending=True)# Your answer here
    #return pd.Series(date_norm.sort_values(by="Parsed date 2", ascending=True).index.values, dtype="int")# Your answer here


date_sorter().sample(50)
#output = date_sorter()
#print("How many to go out of 500:")
#print(sum(pd.isnull(output["Parsed date 2"])))
#output

#date_norm["Original text"].apply(lambda x: re.findall(r"\d{4}", x))

,Original text,Parsed date,Parsed date 2
234,. Patient presents with GAD flareup in Jul 200...,[Jul 2003],2003-07-01
170,"44 y/o MWF, unemployed Decorator, living with ...",[24 Jan 2011],2011-01-24
403,rib resection 10/1981 due to bone tumor in rib\n,[10/1981],1981-10-01
255,sOne week Memorial Psychiatric Hospital Oct 20...,[Oct 2014],2014-10-01
325,.She has however developed severe anxiety and ...,[Jun 2007],2007-06-01
12,8/01/98 Communication with referring physician...,[8/01/98],1998-08-01
159,.The patient brought himself to APS on 26 May ...,[26 May 2001],2001-05-26
154,06 Mar 1974 Primary Care Doctor:\n,[06 Mar 1974],1974-03-06
111,6/10/72 SOS-10 Total Score:\n,[6/10/72],1972-06-10
341,.Nov 2007- Evaluation and treatment at CH ED/A...,[Nov 2007],2007-11-01
